In [1]:
import MySQLdb
import torch
import numpy as np
import matplotlib.pyplot as plt
import stock
import re
from IPython.display import display, Markdown
import ipywidgets as widgets
import xueqiu
import time

In [2]:
#1. 忽略掉不想要的类型
#2. 忽略掉在一年内有连续跌停的
#3. 忽略掉ST
#符合条件的filter返回True
keywords = [r".*证券.*",r".*银行.*",r".*芯片.*",r".*半导体.*",r".*集成电路.*",r".*互联网.*",
            r".*制药.*",r".*生物科技.*",r".*固废.*",r".*通信.*",r".*光电.*",r".*酒.*"]
def descFilter(c):
    for pat in keywords:
        if re.match(pat,c[3]) is not None:
            return True
    return False
def IgnoreCompany(filter=descFilter):
    db = MySQLdb.connect("localhost", "root", "789", "stock", charset='utf8',port=3307 )
    cursor = db.cursor()
    try:
        cursor.execute("""update company set `ignore`=1""")
        db.commit()
    except Exception as e:
        print(e)
        db.rollback()
        
    db.query("""select * from descript where name not like '%ST%' and name not like '%B%'""")
    r = db.store_result()
    companys = r.fetch_row(r.num_rows())
    for c in companys:
        if filter(c):
            try:
                cursor.execute("""update company set `ignore`=0 where code='%s'"""%(c[2]))
                db.commit()
            except Exception as e:
                print(e)
                db.rollback()
    try:
        cursor.execute("""select count(*) from company where `ignore`=0""")
        db.commit()
    except Exception as e:
        print(e)
        db.rollback()
    result = cursor.fetchone()
    db.close()
    print(result)
    
def loadCompany():
    db = MySQLdb.connect("localhost", "root", "789", "stock", charset='utf8',port=3307 )
    db.query("""select * from company where `ignore`=0""")
    r = db.store_result()
    companys = r.fetch_row(r.num_rows())
    db.close()
    return companys

def loadKline(code,n=120):
    db = MySQLdb.connect("localhost", "root", "789", "stock", charset='utf8',port=3307 )
    db.query("""select * from company where code='%s'"""%code)
    r = db.store_result()
    company = r.fetch_row()
    db.query("""select volume,open,high,low,close,macd from kd_xueqiu where id=%s order by date desc limit %d"""%(company[0][0],n))
    r = db.store_result()
    k = r.fetch_row(r.num_rows())
    db.query("""select date from kd_xueqiu where id=%s order by date desc limit %d"""%(company[0][0],n))
    r = db.store_result()
    kdate = r.fetch_row(r.num_rows())
    db.close()
    kline = np.array(k).reshape(-1,6)
    return company,np.flip(k,0),np.flip(kdate,0)

# 使用boll 极限宽带小于0.1来选择股票

In [3]:
#返回头部满足宽带小于wt,长度大于等于nt的位置
def bollWidthHead(bw,bi,ei,wt,nt):
    for i in range(bi,ei-nt):
        head = bw[i:i+nt]
        r = head.max()-head.min()
        if head.max()-head.min()<wt:
            return i+nt
    return -1

#符合条件的返回True,不符合的返回False
#before检查起点迁移
def inBollAisle(k,bi=0,ut=0.2,wt=0.1,nt=15):
    bo20 = stock.bollLineK(k,20)
    lw = (bo20[:,2]-bo20[:,0])/bo20[:,1]
    lwn = []
    for i in range(len(lw)-bi-1,0,-1):
        if lw[i]<ut:
            lwn.append(lw[i])
        else:
            break
    lws = np.array(lwn)
    if len(lws)>=nt and k[-1,4]>bo20[-1,1]:
        #在头部必须要有一段比较窄的头部
        if bollWidthHead(lw,len(lw)-len(lws),len(lw),wt,nt)>=0:
            return True
    return False

def storeSelectCompany(blist):
    db = MySQLdb.connect("localhost", "root", "789", "stock", charset='utf8',port=3307 )
    cursor = db.cursor()
    try:
        cursor.execute("""delete from company_select2 where source=1""")
        db.commit()
    except Exception as e:
        print(e)
        db.rollback()
    for c in blist:
        try:
            cursor.execute("""insert ignore into company_select2 values ('%s','%s',1)"""%(c[0],c[1]))
            db.commit()
        except Exception as e:
            print(e)
            db.rollback()        
    db.close()
    
def ouputSelectCompany(str,blist,i=-1):
    display(Markdown("""## %s %d"""%(str,len(blist))))
    for b in blist:
        if i==-1 or b[2]==i:
            display(Markdown("""[%s %s](https://xueqiu.com/S/%s)"""%(b[0],b[1],b[0])))
def showSelectCompany(blist):
    ouputSelectCompany('通道中',blist,0)
    ouputSelectCompany('已打开',blist,1)
    
def selectList():
    companys = loadCompany()
    result = []
    for com in companys:
        c,kline,kdate = loadKline(com[1])
        isinc = inBollAisle(kline)
        isinb = inBollAisle(kline,1)
        if isinc:
            result.append([com[1],com[2],0])
        elif isinb:
            result.append([com[1],com[2],1])
    return result

In [4]:
bol = selectList()
showSelectCompany(bol)

## 通道中 481

[SZ002229 鸿博股份](https://xueqiu.com/S/SZ002229)

[SZ002585 双星新材](https://xueqiu.com/S/SZ002585)

[SH601968 宝钢包装](https://xueqiu.com/S/SH601968)

[SZ002117 东港股份](https://xueqiu.com/S/SZ002117)

...

[SZ000514 渝开发](https://xueqiu.com/S/SZ000514)

[SZ300770 新媒股份](https://xueqiu.com/S/SZ300770)

[SZ300386 飞天诚信](https://xueqiu.com/S/SZ300386)

[SH600511 国药股份](https://xueqiu.com/S/SH600511)

[SZ000596 古井贡酒](https://xueqiu.com/S/SZ000596)

# 对在BOLL通道内的进行及时检测
1. 成交量比平均值或者上一个交易日有明显放大
2. 股价即时有明显增长

In [6]:
def watch15(code):
    b,json = xueqiu.xueqiuK15(code)
    today_price = 0
    yesterday_price = 0
    today_volume = 0
    yesterday_volume = 0
    today_vlist = []
    yestorday_vlist = []
    today_plist = []
    if b :
        days = set()
        items = json['data']['item']
        c = time.localtime()
        mins = []
        for i in range(-1,-len(items),-1):
            ts = time.localtime(items[i][0]/1000.)
            if c.tm_mday == ts.tm_mday: #是今天
                days.add(ts.tm_mday)
                today_vlist.append(items[i][1]) #今天第n个15分钟的交易量
                today_plist.append(items[i][5]) #今天第n个15分钟的股价
                today_volume += items[i][1] #累计值
                if today_price==0:
                    today_price = items[i][5]
                mins.append(ts.tm_hour*60+ts.tm_min)
            elif ts.tm_hour*60+ts.tm_min in mins:
                days.add(ts.tm_mday)
                yestorday_vlist.append(items[i][1]) #昨天同期的交易量
                yesterday_volume += items[i][1]
                if yesterday_price==0:
                    yesterday_price = items[i][5]
    else:
        print(json)
    if today_volume==0 or yesterday_volume ==0:
        return 0,0
    #不符合条件返回0
    #1. 成交量比平均值或者上一个交易日有明显放大
    #2. 股价即时有明显增长
    c,k,d = loadKline(code,60)
    ma20_volume = stock.ma(k[:,0],20)[-1]
    yesterday_total = k[-1,0]
    if today_volume/yesterday_volume>1.5 and k[-1,4]<items[-1][4]: #并且要股价有所上升
        return 1,today_volume/yesterday_volume
    elif (yesterday_total*today_volume)/(ma20_volume*yesterday_volume)>1.5 and k[-1,4]<items[-1][4]: #成交量同期放大1.5倍,并且要股价有所上升
        #print(c[0][2],yesterday_total,ma20_volume,today_volume,yesterday_volume,(yesterday_total*today_volume)/(ma20_volume*yesterday_volume))
        return 2,(yesterday_total*today_volume)/(ma20_volume*yesterday_volume)
    elif len(today_plist)>1 and (today_plist[0]-today_plist[1])/today_plist[1] > 0.01: #15分钟股价上涨超过1%
        return 3,(today_plist[0]-today_plist[1])/today_plist[1]
    else:
        return 0,0
def watch15Boll(ls):
    bls1 = []
    bls2 = []
    bls3 = []

    progress = widgets.IntProgress(
        value=0,
        min=0,
        max=10,
        step=1,
        description='正在搜寻:',
        bar_style='', # 'success', 'info', 'warning', 'danger' or ''
        orientation='horizontal'
    )
    output = widgets.Output()
    display(output,progress)
    i = 0
    with output:
        for c in ls:
            t,v = watch15(c[0])
            if t==1:
                bls1.append([c[0],"%s %.2f"%(c[1],v),v])
            elif t==2:
                bls2.append([c[0],"%s %.2f"%(c[1],v),v])
            elif t==3:
                bls3.append([c[0],"%s %.2f%%"%(c[1],100*v),100*v])
            progress.value = 10*i/(len(ls))
            i+=1
        progress.value = 10
    bls1 = sorted(bls1,key=lambda c:c[2],reverse = True)
    bls2 = sorted(bls2,key=lambda c:c[2],reverse = True)
    bls3 = sorted(bls3,key=lambda c:c[2],reverse = True)
    ouputSelectCompany('成交量放大',bls1)
    ouputSelectCompany('成交量同比放大',bls2)
    ouputSelectCompany('股价短期上涨',bls3)

In [8]:
watch15Boll(bol)

Output()

IntProgress(value=0, description='正在搜寻:', max=10)

## 成交量放大 51

[SH603888 新华网 9.58](https://xueqiu.com/S/SH603888)

[SZ300766 每日互动 8.49](https://xueqiu.com/S/SZ300766)

...

[SZ300300 汉鼎宇佑 1.21%](https://xueqiu.com/S/SZ300300)

[SH600171 上海贝岭 1.18%](https://xueqiu.com/S/SH600171)

[SZ002148 北纬科技 1.12%](https://xueqiu.com/S/SZ002148)

[SH601788 光大证券 1.11%](https://xueqiu.com/S/SH601788)

[SH600728 佳都科技 1.04%](https://xueqiu.com/S/SH600728)